In [1]:
import requests


France_weather = requests.get('https://public.opendatasoft.com/api/records/1.0/search/?dataset=donnees-synop-essentielles-omm&q=date%3A%5B2015-12-31T23%3A00%3A00Z+TO+2022-12-31T22%3A59%3A59Z%5D&rows=-1&sort=date&facet=date&facet=nom&facet=temps_present&facet=libgeo&facet=nom_epci&facet=nom_dept&facet=nom_reg')
France_weather_data = France_weather.json()


In [2]:
France_weather_data

{'nhits': 1182747,
 'parameters': {'dataset': 'donnees-synop-essentielles-omm',
  'q': 'date:[2015-12-31T23:00:00Z TO 2022-12-31T22:59:59Z]',
  'rows': 10000,
  'start': 0,
  'sort': ['date'],
  'facet': ['date',
   'nom',
   'temps_present',
   'libgeo',
   'nom_epci',
   'nom_dept',
   'nom_reg'],
  'format': 'json',
  'timezone': 'UTC'},
 'records': [{'datasetid': 'donnees-synop-essentielles-omm',
   'recordid': 'd65044a8673d69d9d8049e129c8bf77ec1d0f326',
   'fields': {'hbas': 800.0,
    'rafper': 4.2,
    'n': 90.0,
    'code_dep': '2A',
    'libgeo': 'Ajaccio',
    'pmer': 103210,
    't': 283.75,
    'code_epci': '242010056',
    'ww': '0',
    'numer_sta': '07761',
    'raf10': 4.2,
    'hnuage1': 720.0,
    'code_reg': '94',
    'codegeo': '2A004',
    'rr3': 0.0,
    'coordonnees': [41.918, 8.792667],
    'tc': 10.600000000000023,
    'per': -10.0,
    'nom_dept': 'Corse-du-Sud',
    'nom_reg': 'Corse',
    'mois_de_l_annee': 12,
    'td': 282.05,
    'temps_present': 'On n’a 

In [79]:


import datetime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import *

engine = create_engine('sqlite:///db.sqlite')
Base = declarative_base()

Session = sessionmaker(bind=engine)
session = Session()

class Weather(Base):
    __tablename__ = "Weather"

    recordid = Column(String, nullable=True, primary_key=True)
    tc = Column(REAL)
    nom = Column(String, nullable=True)
    date = Column(String, nullable=True)
    lat = Column(REAL)
    lon = Column(REAL)
    codegeo = Column(String, nullable=True)
    nom_reg = Column(String, nullable=True)
    nom_dept = Column(String, nullable=True)

Base.metadata.create_all(engine)
    

In [80]:
now = datetime.datetime(2021, 12, 31, 0, 0)
y = now.year
while y >=2016:
    date_minus_15 = now - datetime.timedelta(days=15)
    d = now.day
    m = now.month
    y = now.year
    d_m_15 = date_minus_15.day
    m_m_15 = date_minus_15.month
    y_m_15 = date_minus_15.year

    response = requests.get(f'https://public.opendatasoft.com/api/records/1.0/search/?dataset=donnees-synop-essentielles-omm&q=date%3A%5B{str(y_m_15)}-{str(m_m_15)}-{str(d_m_15)}T23%3A00%3A00Z+TO+{str(y)}-{str(m)}-{str(d)}T22%3A59%3A59Z%5D&rows=-1&sort=date&facet=date&facet=nom&facet=temps_present&facet=libgeo&facet=nom_epci&facet=nom_dept&facet=nom_reg').json()
    for i in response['records']:
        try:
            tc = i['fields']['tc']
        except:
            tc = None
        
        try :
            codegeo = i['fields']['codegeo']
        except :
            codegeo = None

        try :
            nom_reg = i['fields']['nom_reg']
        except :
            nom_reg = None

        try :
            nom_dept = i['fields']['nom_dept']
        except :
            nom_dept = None

        session.add(Weather(recordid = i['recordid'],
                            tc = tc,
                            nom = i['fields']['nom'],
                            date = i['fields']['date'],
                            lat = i['fields']['coordonnees'][0],
                            lon = i['fields']['coordonnees'][1],
                            codegeo = codegeo,
                            nom_reg = nom_reg,
                            nom_dept = nom_dept,
                            ))
        session.commit()
    
    print(f"now : {now.day}/{now.month}/{now.year} \t now-15 days : {date_minus_15.day}/{date_minus_15.month}/{date_minus_15.year}  ... OK"  )
    now = date_minus_15


now : 31/12/2021 	 now-15 days : 16/12/2021  ... OK
now : 16/12/2021 	 now-15 days : 1/12/2021  ... OK
now : 1/12/2021 	 now-15 days : 16/11/2021  ... OK
now : 16/11/2021 	 now-15 days : 1/11/2021  ... OK
now : 1/11/2021 	 now-15 days : 17/10/2021  ... OK
now : 17/10/2021 	 now-15 days : 2/10/2021  ... OK
now : 2/10/2021 	 now-15 days : 17/9/2021  ... OK
now : 17/9/2021 	 now-15 days : 2/9/2021  ... OK
now : 2/9/2021 	 now-15 days : 18/8/2021  ... OK
now : 18/8/2021 	 now-15 days : 3/8/2021  ... OK
now : 3/8/2021 	 now-15 days : 19/7/2021  ... OK
now : 19/7/2021 	 now-15 days : 4/7/2021  ... OK
now : 4/7/2021 	 now-15 days : 19/6/2021  ... OK
now : 19/6/2021 	 now-15 days : 4/6/2021  ... OK
now : 4/6/2021 	 now-15 days : 20/5/2021  ... OK
now : 20/5/2021 	 now-15 days : 5/5/2021  ... OK
now : 5/5/2021 	 now-15 days : 20/4/2021  ... OK
now : 20/4/2021 	 now-15 days : 5/4/2021  ... OK
now : 5/4/2021 	 now-15 days : 21/3/2021  ... OK
now : 21/3/2021 	 now-15 days : 6/3/2021  ... OK
now : 

In [78]:
Weather.__table__.drop(engine)

In [63]:
date_minus_15 = now - datetime.timedelta(days=15)
d = now.day
m = now.month
y = now.year
d_m_15 = date_minus_15.day
m_m_15 = date_minus_15.month
y_m_15 = date_minus_15.year

response = requests.get(f'https://public.opendatasoft.com/api/records/1.0/search/?dataset=donnees-synop-essentielles-omm&q=date%3A%5B{str(y_m_15)}-{str(m_m_15)}-{str(d_m_15)}T23%3A00%3A00Z+TO+{str(y)}-{str(m)}-{str(d)}T22%3A59%3A59Z%5D&rows=-1&sort=date&facet=date&facet=nom&facet=temps_present&facet=libgeo&facet=nom_epci&facet=nom_dept&facet=nom_reg')

In [34]:
response.json()

{'nhits': 6717,
 'parameters': {'dataset': 'donnees-synop-essentielles-omm',
  'q': 'date:[2022-12-5T23:00:00Z TO 2022-12-20T22:59:59Z]',
  'rows': 10000,
  'start': 0,
  'sort': ['date'],
  'facet': ['date',
   'nom',
   'temps_present',
   'libgeo',
   'nom_epci',
   'nom_dept',
   'nom_reg'],
  'format': 'json',
  'timezone': 'UTC'},
 'records': [{'datasetid': 'donnees-synop-essentielles-omm',
   'recordid': 'd65044a8673d69d9d8049e129c8bf77ec1d0f326',
   'fields': {'hbas': 800.0,
    'rafper': 4.2,
    'n': 90.0,
    'code_dep': '2A',
    'libgeo': 'Ajaccio',
    'pmer': 103210,
    't': 283.75,
    'code_epci': '242010056',
    'ww': '0',
    'numer_sta': '07761',
    'raf10': 4.2,
    'hnuage1': 720.0,
    'code_reg': '94',
    'codegeo': '2A004',
    'rr3': 0.0,
    'coordonnees': [41.918, 8.792667],
    'tc': 10.600000000000023,
    'per': -10.0,
    'nom_dept': 'Corse-du-Sud',
    'nom_reg': 'Corse',
    'mois_de_l_annee': 12,
    'td': 282.05,
    'temps_present': 'On n’a pas 